In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
url = "https://www.youtube.com/intl/en-GB/howyoutubeworks/policies/community-guidelines/"
driver.get(url)
time.sleep(3)

page_source = driver.page_source
driver.quit()
soup = BeautifulSoup(page_source, "html.parser")

links = [a["href"] for a in soup.find_all("a", href=True)]

guideline_links = [i for i in links if "answer" in i]

In [2]:
for i in guideline_links:
    print(i)

https://support.google.com/youtube/answer/3399767?hl=en&ref_topic=9282365
https://support.google.com/youtube/answer/2801947?hl=en&ref_topic=9282365
https://support.google.com/youtube/answer/9054257?hl=en&ref_topic=9282365
https://support.google.com/youtube/answer/2801973?hl=en&ref_topic=9282365
https://support.google.com/youtube/answer/9713446?hl=en&ref_topic=9282365
https://support.google.com/youtube/answer/2801981?hl=en&ref_topic=9282365
https://support.google.com/youtube/answer/2801999?hl=en&ref_topic=9282679
https://support.google.com/youtube/answer/9229980?hl=en&ref_topic=9282679
https://support.google.com/youtube/answer/2802002?hl=en&ref_topic=9282679
https://support.google.com/youtube/answer/2802245?hl=en&ref_topic=9282679
https://support.google.com/youtube/answer/10072685?hl=en&ref_topic=9282679
https://support.google.com/youtube/answer/2802268?hl=en&ref_topic=9282436
https://support.google.com/youtube/answer/2801964?hl=en&ref_topic=9282436
https://support.google.com/youtube/an

In [4]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
doc=[]
for i in guideline_links:
    loader=WebBaseLoader(web_path=i,bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_="article page")))
    doc.extend(loader.load())

In [5]:
doc

[Document(metadata={'source': 'https://support.google.com/youtube/answer/3399767?hl=en&ref_topic=9282365'}, page_content=" Fake engagement policy\nThe safety of our creators, viewers, and partners is our highest priority. We look to each of you to help us protect this unique and vibrant community. It’s important you understand our Community Guidelines, and the role they play in our shared responsibility to keep YouTube safe. Take the time to carefully read the policy below. You can also check out this page for a full list of our guidelines.\n\n\nYouTube doesn’t allow anything that artificially increases the number of views, likes, comments, or other metrics either by using automatic systems or serving up videos to unsuspecting viewers. Also, content that solely exists to incentivize viewers for engagement (views, likes, comments, etc) is prohibited.\nContent and channels\xa0that don't follow this policy may be terminated and removed from YouTube.\nImportant:\xa0\xa0If you hire someone 

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
document=text_splitter.split_documents(doc)
document

[Document(metadata={'source': 'https://support.google.com/youtube/answer/3399767?hl=en&ref_topic=9282365'}, page_content='Fake engagement policy\nThe safety of our creators, viewers, and partners is our highest priority. We look to each of you to help us protect this unique and vibrant community. It’s important you understand our Community Guidelines, and the role they play in our shared responsibility to keep YouTube safe. Take the time to carefully read the policy below. You can also check out this page for a full list of our guidelines.'),
 Document(metadata={'source': 'https://support.google.com/youtube/answer/3399767?hl=en&ref_topic=9282365'}, page_content="YouTube doesn’t allow anything that artificially increases the number of views, likes, comments, or other metrics either by using automatic systems or serving up videos to unsuspecting viewers. Also, content that solely exists to incentivize viewers for engagement (views, likes, comments, etc) is prohibited.\nContent and channe

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") 

db = Chroma.from_documents(documents=document, embedding=embeddings, persist_directory="../chroma_db")

db.persist()



C:\Users\23ary\AppData\Local\Temp\ipykernel_3556\2942246766.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
